In [ ]:
import requests
import time

# Define the base URL for the API
base_url = 'https://li.quest/v1'

# Fetch all connections with the allowExchanges parameter so not to limit the toChain or fromChain
connections_response = requests.get(f'{base_url}/connections', params={'allowExchanges': 'all'})
connections = connections_response.json()['connections']

# Define the amount of USDC to swap
amount = '1000000000000'  # 1000 USDC, assuming 6 decimal places

print(connections_response.json())

# Define the address initiating the swaps
from_address = '0xD5AEFc1fED909DA9a5409594d758E3bdd055634c'  # Metamask Main Test

# List to store the swap costs
swap_costs = []

# Dictionary to store bridge recommendation counts
bridge_counts = {}
no_bridge_count = 0

# Function to update bridge counts
def update_bridge_counts(bridge_name):
    if bridge_name:
        bridge_counts[bridge_name] = bridge_counts.get(bridge_name, 0) + 1
    else:
        global no_bridge_count
        no_bridge_count += 1

# Iterate over all connections
for connection in connections:
    from_tokens = connection['fromTokens']
    to_tokens = connection['toTokens']

    # Check if the connection involves USDC and USDT
    for from_token in from_tokens:
        if from_token['symbol'] == 'USDC':
            for to_token in to_tokens:
                if to_token['symbol'] == 'USDT':
                    # Fetch a quote for the swap
                    quote_response = requests.get(f'{base_url}/quote', params={
                        'fromChain': connection['fromChainId'],
                        'toChain': connection['toChainId'],
                        'fromToken': from_token['address'],
                        'toToken': to_token['address'],
                        'fromAmount': amount,
                        'fromAddress': from_address
                    })

                    quote = quote_response.json()

                    # Update bridge counts based on the main tool
                    update_bridge_counts(quote.get('tool'))

                    # Check if the 'estimate' key exists in the quote
                    if 'estimate' in quote:
                        # Extract the relevant cost information from the quote
                        gas_cost = '0'
                        exchange_cost = '0'

                        if 'gasCosts' in quote['estimate'] and quote['estimate']['gasCosts']:
                            gas_cost = quote['estimate']['gasCosts'][0]['amount']

                        if 'exchangeCosts' in quote['estimate'] and quote['estimate']['exchangeCosts']:
                            exchange_cost = quote['estimate']['exchangeCosts'][0]['amount']

                        total_cost = float(gas_cost) + float(exchange_cost)

                        # Append the swap cost to the list
                        swap_costs.append({
                            'fromChain': connection['fromChainId'],
                            'toChain': connection['toChainId'],
                            'gasCost': gas_cost,
                            'exchangeCost': exchange_cost,
                            'totalCost': total_cost
                        })

                    time.sleep(5)  # Sleep for 5 seconds between requests

# Sort the swap costs in descending order based on the total cost
sorted_swap_costs = sorted(swap_costs, key=lambda x: x['totalCost'], reverse=True)

# Print the sorted swap costs
for cost in sorted_swap_costs:
    print(f"Swap from chain {cost['fromChain']} to chain {cost['toChain']}:")
    print(f"Gas Cost: {cost['gasCost']}")
    print(f"Exchange Cost: {cost['exchangeCost']}")
    print(f"Total Cost: {cost['totalCost']}")
    print("---")

# Print the bridge recommendation counts
print("Bridge Recommendation Counts:")
for bridge, count in bridge_counts.items():
    print(f"{bridge}: {count} times")
print(f"No bridge was recommended {no_bridge_count} times.")

{'connections': [{'fromChainId': 1, 'toChainId': 137, 'fromTokens': [{'address': '0xdAC17F958D2ee523a2206206994597C13D831ec7', 'chainId': 1, 'symbol': 'USDT', 'decimals': 6, 'name': 'USDT', 'coinKey': 'USDT', 'logoURI': 'https://raw.githubusercontent.com/trustwallet/assets/master/blockchains/ethereum/assets/0xdAC17F958D2ee523a2206206994597C13D831ec7/logo.png', 'priceUSD': '0.999475'}, {'address': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48', 'chainId': 1, 'symbol': 'USDC', 'decimals': 6, 'name': 'USD Coin', 'coinKey': 'USDC', 'logoURI': 'https://raw.githubusercontent.com/trustwallet/assets/master/blockchains/ethereum/assets/0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48/logo.png', 'priceUSD': '1'}], 'toTokens': [{'address': '0xc2132D05D31c914a87C6611C10748AEb04B58e8F', 'chainId': 137, 'symbol': 'USDT', 'decimals': 6, 'name': 'USDT', 'coinKey': 'USDT', 'logoURI': 'https://raw.githubusercontent.com/trustwallet/assets/master/blockchains/ethereum/assets/0xdAC17F958D2ee523a2206206994597C13D831e